# Advice for SQL Interviews Questions: Focus on Fundamentals

1. Know basic SQL operations inside out (`SELECT`, `WHERE` various types of `JOIN`s, `GROUP BY`, `ORDER BY`)
2. Use temporary tables to break computation into smaller pieces
3. Practice, practice, practice!


## Setup

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('example.db')

In [2]:
conn.execute('''DROP TABLE t1;''')
conn.execute('''DROP TABLE t2;''')
conn.commit()
conn.execute('''CREATE TABLE t1(id INTEGER, name TEXT, date DATE, status TEXT);''')
conn.execute('''CREATE TABLE t2(id INTEGER, date DATE, price REAL);''')
conn.commit()

In [3]:
from random import randint, uniform
from datetime import date, timedelta

names = ['Bob', 'Alice', 'Mary', 'Sue', 'John', 'Peter', 'Jenny', 'Cat', 'Steve', 'Will']

for y in range(10):
    for id, name in enumerate(names):
        entry_date = date.today() - timedelta(days=randint(1, 1000))
        status = randint(0, 1)
        conn.execute('INSERT INTO t1 VALUES(?, ?, ?, ?)', (id, name, entry_date, status)) 
        conn.execute('INSERT INTO t2 VALUES(?, ?, ?)', (id, entry_date, round(uniform(1, 1000), 2))) 
        
conn.commit()

In [4]:
pd.read_sql_query("SELECT * FROM t1", conn)

,id,name,date,status
0,0,Bob,2020-02-17,1
1,1,Alice,2018-10-11,0
2,2,Mary,2018-07-06,1
3,3,Sue,2019-11-02,0
4,4,John,2018-11-02,0
...,...,...,...,...
95,5,Peter,2019-07-02,1
96,6,Jenny,2018-11-17,0
97,7,Cat,2018-12-11,1
98,8,Steve,2020-09-28,0


In [5]:
pd.read_sql_query("SELECT * FROM t2", conn)

,id,date,price
0,0,2020-02-17,94.88
1,1,2018-10-11,284.31
2,2,2018-07-06,123.91
3,3,2019-11-02,714.12
4,4,2018-11-02,931.29
...,...,...,...
95,5,2019-07-02,607.92
96,6,2018-11-17,90.37
97,7,2018-12-11,991.51
98,8,2020-09-28,440.40


## Example 1: Select unique names in reverse order

In [6]:
query = """
SELECT DISTINCT(name)
FROM t1
ORDER BY name DESC
"""
pd.read_sql_query(query, conn)

,name
0,Will
1,Sue
2,Steve
3,Peter
4,Mary
5,John
6,Jenny
7,Cat
8,Bob
9,Alice


## Example 2: Find total spend of each person since 2021-01-01

In [7]:
query = """
SELECT name, SUM(price)
FROM t1 JOIN t2 USING(id)
WHERE t1.date >= '2021-01-01'
GROUP BY name
"""
pd.read_sql_query(query, conn)

query = """
WITH aa AS (
    SELECT id, name
    FROM t1
), bb AS (
    SELECT *
    FROM t2
    WHERE date >= '2021-01-01'
)
SELECT name, SUM(price)
FROM aa LEFT JOIN bb USING(id)
GROUP BY name
ORDER BY SUM(price) DESC
"""
pd.read_sql_query(query, conn)

,name,SUM(price)
0,Peter,17273.4
1,Will,6484.7
2,Steve,3691.3
3,Sue,2914.1
4,Mary,NaN
5,John,NaN
6,Jenny,NaN
7,Cat,NaN
8,Bob,NaN
9,Alice,NaN


### Exercises

* Make sure all names are listed, even if they didn't spend anything